# Asymptotic expansion of the energy

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\order}[2][1]{#2^{(#1)}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In this chapter, we seek an asymptotic expansion of the energy along the bifurcated branch. More precisely, the following quantity will be expanded at the critical point

$$
\Delta\E(\eta) = \E[u(\eta), \lambda(\eta)] - \E[u^\ast \circ \lambda(\eta), \lambda(\eta)],
$$

where $\eta$ is an arbitrary parametrization of the bifurcated branch ($\eta = 0$ at the critical point).

The starting point is the asymptotic expansions of $\lambda(\eta)$ and $u(\eta)$ obtained in @sec-20230102030224, which are reproduced below

$$
\lambda(\eta) = \order[1]{\lambda} \, \eta + \tfrac{1}{2} \order[2]{\lambda} \, \eta^2 + o(\eta^2)
$$

and

$$
\begin{aligned}[b]
u(\eta) ={}& u^\ast[\lambda(\eta)] + \eta \, \order[1]{\xi_i} \, v_i\\
&+ \tfrac{1}{2} \eta^2 \, \bigl( \order[2]{\xi_i} \, v_i
+ \order[1]{\xi_i} \, \order[1]{\xi_j} \, w_{ij}  + 2\order[1]{\lambda} \, \order[1]{\xi_i} \, w_{i\lambda} \bigr) + o(\eta^2).
\end{aligned}
$$ {#eq-20230402141825}

The asymptotic expansion of the energy results from plugging the above expansions into the general expression of the energy that was defined in @sec-20230402140105.

In [1]:
import json
import lsk

from sympy import *

with open("setting-up_the_computational_stage.out.json", "r") as f:
    data = json.load(f)

We first define the asymptotic expansion of $\lambda$ w.r.t. the (unspecified) parameter $\eta$.

In [2]:
η, λ1, λ2, λ3, λ4 = symbols(r"\eta {\order[1]{\lambda}} {\order[2]{\lambda}} {\order[3]{\lambda}} {\order[4]{\lambda}}")
λ = λ1 * η + λ2 * η**2 / 2 + λ3 * η**3 / 6 + λ4 * η**4 / 24

In [3]:
#| code-fold: true
lsk.display_latex_equation(r"\lambda(\eta)", λ)

<IPython.core.display.Math object>

::: {.callout-note}
In the previous chapters, we have derived the asymptotic expansions of $\lambda$, $u^\ast$ and $u$ up to fourth-order in $\eta$. It will be shown below that these expansion lead to an expansion of $\Delta\E$ up to **fourth-order** terms in $\eta$.

For the sake of consistency, expansions of $\lambda$, $u^\ast$ and $u$ will be postulated up to fourth-order: it will be finally checked that the fourth-order expansion of $\Delta\E$ refers only to: $\order[1]{\lambda}$, $\order[2]{\lambda}$, $w_{i\lambda}$ and $w_{ij}$ (which are all known quantities).
:::

The fundamental branch $u^\ast(\lambda)$ was previously defined as an asymptotic expansion w.r.t. $\lambda$.

In [4]:
u_star_of_λ = eval(data["u_star"])

In [5]:
#| code-fold: true
lsk.display_latex_equation(r"u^\ast(\lambda)", u_star_of_λ)

<IPython.core.display.Math object>

 We now define the fundamental branch as an asymptotic expansion w.r.t. $\eta$, keeping only fourth-order terms.

In [6]:
u_star = u_star_of_λ.subs(Symbol("\lambda"), λ + O(η**5)).expand().removeO()

In [7]:
#| code-fold: true
lsk.display_latex_equation(r"u^\ast(\eta)", u_star)

<IPython.core.display.Math object>

Similarly, we define the asymptotic expansion (@eq-20230402141825) of $u$; note that the quantities $\order[3]{\tilde{u}}$ and $\order[4]{\tilde{u}}$ are unknown (and will in fact disappear from the derivation). 

In [8]:
u = (u_star
     + η * Symbol(r"\xi_i^{(1)}v_i") 
     + η**2 / 2 * (Symbol(r"\xi_j^{(2)}v_j")
                   + Symbol(r"\xi_k^{(1)}\xi_l^{(1)}w_{kl}")
                   + 2 * λ1 * Symbol(r"\xi_m^{(1)}w_{m\lambda}"))
     + η**3 / 6 * Symbol(r"{\order[3]{\tilde{u}}}")
     + η**4 / 24 * Symbol(r"{\order[4]{\tilde{u}}}")).expand()

In [9]:
#| code-fold: true
lsk.display_latex_equation(r"u(\eta)", u)

<IPython.core.display.Math object>

We then use these expansions to evaluate the energy along the fundamental and bifurcated branches. The general expression of the energy (see @sec-20230402140105) is first retrieved

In [10]:
E_ = eval(data["E"])

In [11]:
#| code-fold: true
lsk.display_latex_equation(r"\E(u ,\lambda)", E_)

<IPython.core.display.Math object>

The energy is evaluated along the **fundamental** branch

In [12]:
E_star = (E_.subs({Symbol("\lambda"): λ,
                   Symbol("u"): u_star}).expand() + O(η**5)).removeO()

In [13]:
#| code-fold: true
lsk.display_latex_equation(r"\E[u^\ast \circ \lambda(\eta), \lambda(\eta)]", E_star)

<IPython.core.display.Math object>

Similarly, the energy is evaluated along the **bifurcated** branch

In [14]:
E = (E_.subs({Symbol("\lambda"): λ,
              Symbol("u"): u}).expand() + O(η**5)).removeO()

In [15]:
#| code-fold: true
lsk.display_latex_equation(r"\E[u(\eta), \lambda(\eta)]", E)

<IPython.core.display.Math object>

The difference is now evaluated.

In [16]:
ΔE = E - E_star

The terms of order 0 and 1 in $\eta$ are identically null:

In [17]:
assert ΔE.coeff(η, 0) == 0
assert ΔE.coeff(η, 1) == 0

The second-order term reads

In [18]:
ΔE2 = ΔE.coeff(η, 2)

In [19]:
#| code-fold: true
lsk.display_latex_equation(r"\order[2]{\Delta\E}", ΔE2)

<IPython.core.display.Math object>

The above expression should be understood as $\frac{1}{2} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_2(v_i, v_j)$, which vanishes since $v_i$ and $v_j$ belong to the kernel of $\E_2$. We now turn to the third-order term

In [20]:
ΔE3 = ΔE.coeff(η, 3)

In [21]:
#| code-fold: true
lsk.display_latex_equation(r"\order[3]{\Delta\E}", ΔE3)

<IPython.core.display.Math object>

Observing that $\E_2(v_i, \bullet) = 0$ and using Eqs. (@eq-20230125062505), (@eq-20230125062510) and (@eq-20230125062336), the above expression reduces to

$$
\begin{aligned}[b]
\order[3]{\Delta\E}
&= \tfrac{1}{6} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \E_3(v_i, v_j, v_k) + \tfrac{1}{2} \, \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, v_j)\\
&= \tfrac{1}{6} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, E_{ijk} + \tfrac{1}{2} \, \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{E}_{ij}\\
&= \tfrac{1}{6} \, \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{E}_{ij}.
\end{aligned}
$$

Finally, the fourth-order term is simplified.

In [22]:
ΔE4 = ΔE.coeff(η, 4)

In [23]:
#| code-fold: true
lsk.display_latex_equation(r"\order[4]{\Delta\E}", ΔE4)

<IPython.core.display.Math object>

Upon simplification of $\E_2(v_i, \bullet) = 0$, the above expression should be understood as follows

$$
\begin{aligned}[b]
\order[4]{\Delta\E} ={}& \tfrac{1}{8} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_2(w_{ij}, w_{kl})
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \E_2(w_{ij}, w_{k\lambda})\\
&+ \tfrac{1}{2} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_2(w_{i\lambda}, w_{j\lambda})
+ \tfrac{1}{4} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[2]{\xi}_k \, \E_3(v_i, v_j, v_k)\\
&+ \tfrac{1}{4} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_3(v_i, v_j, w_{kl})
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \E_3(v_i, v_j, w_{k\lambda})\\
&+ \tfrac{1}{24} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_4(v_i, v_j, v_k, v_l)
+ \tfrac{1}{4} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \ddot{\E}_2(v_i, v_j)\\
&+ \tfrac{1}{4} \order[2]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, v_j)
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[2]{\xi}_j \, \dot{\E}_2(v_i, v_j)\\
&+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \dot{E}_2(v_i, w_{jk})
+ \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, w_{j\lambda})\\
&+ \tfrac{1}{6} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \dot{\E}_3(v_i, v_j, v_k),
\end{aligned}
$$ {#eq-20230402102020}

which simplifies (see proof in @sec-20230402101835)

$$
\begin{aligned}[b]
\order[4]{\Delta\E} ={} & \tfrac{1}{24} \bigl\{ E_{ijkl} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l + 4\order[1]{\lambda} \, \dot{E}_{ijk} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k\\
&+ 6 \bigl[ \bigl( \order[1]{\lambda} \bigr)^2 \, \ddot{E}_{ij} + \order[2]{\lambda} \, \dot{E}_{ij} \bigr] \, \order[1]{\xi}_i \, \order[1]{\xi}_j \bigr\}.
\end{aligned}
$$ {#eq-20230402101651}

To sum up, the following asymptotic expansion has been derived

$$
\begin{aligned}[b]
\E[u(\eta), \lambda(\eta)] ={}& \E[u^\ast \circ \lambda(\eta), \lambda(\eta)]
+ \tfrac{1}{6} \eta^3 \, \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{E}_{ij}\\
&+ \tfrac{1}{24} \eta^4 \, \bigl\{ E_{ijkl} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l + 4\order[1]{\lambda} \, \dot{E}_{ijk} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k\\
&+ 6 \bigl[ \bigl( \order[1]{\lambda} \bigr)^2 \, \ddot{E}_{ij} + \order[2]{\lambda} \, \dot{E}_{ij} \bigr] \, \order[1]{\xi}_i \, \order[1]{\xi}_j \bigr\}.
\end{aligned}
$$

To close this chapter, we evaluate the Hessian of the energy along the bifurcated branch.

In [24]:
E_uu = (eval(data["E_uu"]).subs({Symbol("\lambda"): λ, Symbol("u"): u})  + O(η**3)).expand()

In [25]:
E_uu

\dot{\E}_3*\eta**2*\xi_i^{(1)}v_i*{\order[1]{\lambda}} + \dot{\E}_2*\eta*{\order[1]{\lambda}} + \dot{\E}_2*\eta**2*{\order[2]{\lambda}}/2 + \ddot{\E}_2*\eta**2*{\order[1]{\lambda}}**2/2 + \E_4*\eta**2*\xi_i^{(1)}v_i**2/2 + \E_3*\eta*\xi_i^{(1)}v_i + \E_3*\eta**2*\xi_m^{(1)}w_{m\lambda}*{\order[1]{\lambda}} + \E_3*\eta**2*\xi_k^{(1)}\xi_l^{(1)}w_{kl}/2 + \E_3*\eta**2*\xi_j^{(2)}v_j/2 + \E_2 + O(\eta**3)

In [26]:
#| code-fold: true
lsk.display_latex_equation(r"\E_{,uu}[u(\eta), \lambda(\eta)]", E_uu)

<IPython.core.display.Math object>

The above expression must be understood as follows

$$
\begin{aligned}[b]
\E_{,uu}[u(\eta), \lambda(\eta); \bullet, \bullet] ={} 
& \E_2(\bullet, \bullet) + \eta \, \bigl[ \order[1]{\xi}_i \, \E_3(v_i, \bullet, \bullet) + \order[1]{\lambda} \, \dot{\E}_2(\bullet, \bullet) \bigr]\\
&+ \tfrac{1}{2}\eta^2 \, \bigl[ \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_4(v_i, v_j, \bullet, \bullet)
+ \order[2]{\xi}_i \, \E_3(v_i, \bullet, \bullet) \\
&+ \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_3(w_{ij}, \bullet, \bullet)
+ 2\order[1]{\lambda} \, \order[1]{\xi}_i \, \E_3(w_{i\lambda}, \bullet, \bullet)\\
&+ 2\order[1]{\lambda} \, \order[1]{\xi}_i \, \dot{\E}_3(v_i, \bullet, \bullet)
+ (\order[1]{\lambda})^2 \, \ddot{\E}_2(\bullet, \bullet) + \order[2]{\lambda} \, \dot{\E}_2(\bullet, \bullet)\bigr],
\end{aligned}
$$

or, equivalently

$$
\begin{aligned}[b]
\E_{,uu}[u(\eta), \lambda(\eta); \bullet, \bullet] ={} 
& \E_2(\bullet, \bullet) + \eta \, \bigl[ \E_3(\order[1]{u}, \bullet, \bullet) + \order[1]{\lambda} \, \dot{\E}_2(\bullet, \bullet) \bigr]\\
&+ \tfrac{1}{2}\eta^2 \, \bigl[ \E_4(\order[1]{u}, \order[1]{u}, \bullet, \bullet)
+ \E_3(\order[2]{u}, \bullet, \bullet) \\
&+ 2\order[1]{\lambda} \, \dot{\E}_3(\order[1]{u}, \bullet, \bullet)
+ (\order[1]{\lambda})^2 \, \ddot{\E}_2(\bullet, \bullet) + \order[2]{\lambda} \, \dot{\E}_2(\bullet, \bullet)\bigr],
\end{aligned}
$$ {#eq-20230517061452}

where $\order[1]{u}$ and $\order[2]{u}$ have been defined in @sec-20230102030224, see Eq (@eq-20230411141319).